In [1]:
!pip install torch

In [2]:
import pandas as pd
import os
import librosa
import matplotlib
import numpy as np
from tqdm import  tqdm
from sklearn.preprocessing import LabelEncoder

In [3]:
path = "data/data/audio"
metadata = pd.read_csv("data/data/metadata/data.csv")

In [4]:
len(metadata)

8732

In [5]:
def feature_extract(file):
    audio, sample_rate = librosa.load(file)
    mfccs_features = librosa.feature.mfcc(y= audio, sr = sample_rate)
    mfccs_features = np.mean(mfccs_features.T,axis = 0)
    return mfccs_features

In [6]:
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=feature_extract(file_name)
    extracted_features.append([data,final_class_labels])

0it [00:00, ?it/s]d:\Murtaza\machine learning\audio-classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
3548it [00:35, 116.01it/s]d:\Murtaza\machine learning\audio-classification\.venv\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8319it [01:16, 138.02it/s]d:\Murtaza\machine learning\audio-classification\.venv\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
d:\Murtaza\machine learning\audio-classification\.venv\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [01:20, 108.86it/s]


In [7]:
extracted_features

[[array([-211.93698   ,   62.581207  , -122.81315   ,  -60.745293  ,
          -13.89376   ,  -29.789835  ,   -3.978875  ,   11.682742  ,
           12.963828  ,    8.336421  ,   -6.529819  ,   -2.9961064 ,
          -12.88795   ,    5.8859324 ,   -2.4752254 ,   -4.408312  ,
          -15.014478  ,   -0.94399697,   -2.5799103 ,    6.0982966 ],
        dtype=float32),
  'dog_bark'],
 [array([-417.0052   ,   99.336624 ,  -42.995583 ,   51.073326 ,
            9.853778 ,    7.9696927,   11.197088 ,    1.9291172,
            7.0303974,    4.270228 ,   -3.5847256,   13.890946 ,
           -2.7644935,    7.2725554,    5.534419 ,    1.8692906,
            1.7257473,    7.0386686,   -2.442267 ,    3.4736145],
        dtype=float32),
  'children_playing'],
 [array([-4.5239316e+02,  1.1236253e+02, -3.7578072e+01,  4.3195866e+01,
          8.6318455e+00,  1.5379368e+01,  1.6882149e+01,  1.2330472e+00,
          6.8331223e+00,  3.9001155e+00, -5.1688495e+00,  8.3000450e+00,
          1.9041388e+00

In [8]:
features_df = pd.DataFrame(extracted_features, columns=["feature", "class"])


In [9]:
features_df

,feature,class
0,"[-211.93698, 62.581207, -122.81315, -60.745293...",dog_bark
1,"[-417.0052, 99.336624, -42.995583, 51.073326, ...",children_playing
2,"[-452.39316, 112.36253, -37.57807, 43.195866, ...",children_playing
3,"[-406.47922, 91.1966, -25.043558, 42.78452, 11...",children_playing
4,"[-439.63867, 103.86223, -42.658787, 50.690277,...",children_playing
...,...,...
8727,"[-391.3721, 125.36239, -40.634056, 25.899754, ...",car_horn
8728,"[-339.42715, 76.409096, -35.30988, 41.984035, ...",car_horn
8729,"[-297.31534, 102.09026, -36.75108, 26.816488, ...",car_horn
8730,"[-336.92386, 115.3139, -44.847095, 24.968985, ...",car_horn


In [10]:
X = np.array(features_df["feature"].tolist())
y = np.array(features_df["class"].tolist())

In [11]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.2, random_state =67)

In [13]:
X_train.shape

(6985, 20)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import  Dataset, DataLoader, TensorDataset

In [15]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
device

'cpu'

In [32]:
train_df = TensorDataset(torch.tensor(X_train, dtype= torch.float),torch.tensor(y_train, dtype= torch.long))
test_df = TensorDataset(torch.tensor(X_test, dtype= torch.float),torch.tensor(y_test, dtype= torch.long))

In [33]:
train_dl  = DataLoader(train_df, batch_size = 32, shuffle=True,num_workers=4, pin_memory=True)
test_dl = DataLoader(test_df, batch_size = 32, shuffle=False)

In [34]:
for batch in train_dl:
    X,y = batch
    print(X.shape)
    print(y)
    break

d:\Murtaza\machine learning\audio-classification\.venv\Lib\site-packages\torch\utils\data\dataloader.py:1118: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


torch.Size([32, 20])
tensor([3, 4, 1, 5, 2, 1, 0, 2, 3, 7, 0, 1, 9, 8, 0, 8, 4, 3, 7, 2, 7, 5, 7, 4,
        7, 4, 0, 9, 5, 0, 0, 9])


In [35]:
target_size = features_df["class"].nunique()

In [36]:
class MODEL(nn.Module):
    def __init__(self, target_size):
        super().__init__()
        
        self.fc1 = nn.Linear(20, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, target_size)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))
        x = self.fc4(x)
        return x


In [37]:
model = MODEL(target_size).to(device)
model

MODEL(
  (fc1): Linear(in_features=20, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)

In [38]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)
loss_fn =nn.CrossEntropyLoss()  

In [ ]:
for epoch in range(39):
    model.train()
    total_loss = 0

    for x, y in train_dl:
        optimizer.zero_grad()      
        out = model(x)
        loss = loss_fn(out, y) 
        loss.backward()            
        optimizer.step()           
        
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 1221.6744
Epoch 2, Loss: 926.7309
Epoch 3, Loss: 783.2962
Epoch 4, Loss: 705.0332
Epoch 5, Loss: 651.9295
Epoch 6, Loss: 620.9122
Epoch 7, Loss: 591.7057
Epoch 8, Loss: 571.5773
Epoch 9, Loss: 565.2031
Epoch 10, Loss: 549.0191
Epoch 11, Loss: 542.3404
Epoch 12, Loss: 532.3635
Epoch 13, Loss: 525.4740
Epoch 14, Loss: 521.1320
Epoch 15, Loss: 517.9141
Epoch 16, Loss: 511.6810
Epoch 17, Loss: 508.7562
Epoch 18, Loss: 505.7772
Epoch 19, Loss: 505.7095
Epoch 20, Loss: 501.2597
Epoch 21, Loss: 498.0481
Epoch 22, Loss: 495.8110
Epoch 23, Loss: 492.4518
Epoch 24, Loss: 491.3517
Epoch 25, Loss: 488.1420
Epoch 26, Loss: 488.0762
Epoch 27, Loss: 487.5437
Epoch 28, Loss: 484.6087
Epoch 29, Loss: 483.4895
Epoch 30, Loss: 479.2444
Epoch 31, Loss: 478.6397
Epoch 32, Loss: 474.2891
Epoch 33, Loss: 473.4842
Epoch 34, Loss: 474.9677
Epoch 35, Loss: 473.4952
Epoch 36, Loss: 471.6898
Epoch 37, Loss: 467.5601
Epoch 38, Loss: 469.7015
Epoch 39, Loss: 468.8033


SyntaxError: 'return' outside function (1904987314.py, line 16)